Inspiration from https://github.com/cran/RadOnc/blob/master/R/read.DVH.R

And http://stackoverflow.com/questions/3914454/python-how-to-loop-through-blocks-of-lines

In [138]:
%load_ext watermark
%watermark -a 'Robin Cole' -u -d -v -p numpy,pandas,matplotlib

Robin Cole 
last updated: 2016-10-04 

CPython 3.5.2
IPython 4.2.0

numpy 1.11.1
pandas 0.18.1
matplotlib 1.5.1


In [95]:
import numpy as np
import itertools
import json
import re

In [273]:
file = 'case1_AAA.txt'
file_header =  open(file, 'r').readlines()[:15]
file_body =    open(file, 'r').readlines()[16:]  # first 16 lines are header, for some reason was not opening header

plan = {}   # to contain all file plan info

# parse the header here
for line in file_header:
    if line.replace(u'\ufeff', '').startswith('Patient Name'):
        plan['Patient Name'] = line.replace(u'\ufeff', '').split(':')[1].rstrip()
    elif line.startswith('Patient ID'):
        plan['Patient ID'] = line.split(':')[1].rstrip() 
    elif line.startswith('Plan:'):
        plan['Plan'] = line.split(':')[1].rstrip() 
    elif line.startswith('Prescribed dose [Gy]'):
        plan['Prescribed dose [Gy]'] = line.split(':')[1].rstrip()
    else:
        pass

#############################

## get all structures
Structures = []   # create list 
print(len(file_body))
for i, line in enumerate(file_body): 
    matchObj = re.match( r'^Structure: (.*?$)', line, re.M|re.X)
    if matchObj:
        Structures.append(matchObj.group(1))
plan['Structures'] = Structures  

#############################

plan

33200


{'Patient ID': ' Case1_AAA',
 'Patient Name': ' Acuros lung evaluation, RC Physics',
 'Plan': ' Case1_AAA',
 'Prescribed dose [Gy]': ' 55.000',
 'Structures': [' BODY_P',
  ' GTV_P',
  ' Heart_',
  ' Foramen',
  ' Foramen+5mm_P',
  ' Oesophagus_P',
  ' Foramen+3mm_P',
  ' Trachea_P',
  ' Bronchial tree_P',
  ' L Brachial plex_',
  ' Rt Brachial plex',
  ' Airway avoidance',
  ' PTV CHEST_P',
  ' Lt_Lung_P',
  ' PTV ring 2-3cm_P',
  ' Rt_Lung_P',
  ' Both Lungs-ITV_P',
  ' skin_P',
  ' PseudoPTV-ITV_P',
  ' Pseudo ITV_P',
  ' ribs_P',
  ' Pseudo Ring_P',
  ' Pseudo Ribs_P',
  ' Dose 100[%]_P',
  ' Dose 50[%]_P']}

In [319]:
i = 0
for key, Structure_group in itertools.groupby(file_body, lambda line: line.startswith('Structure:')):  # group on Structure
    print(i)    
    for key, sub_group in itertools.groupby(Structure_group, lambda line: line=='\n'):                 # Group on empty lines
        if not key:
            for i, line in enumerate(sub_group):
                if line.startswith('Relative dose'):
                    data = []   # create a list so can use the append . np arrays must be init with length
                    for line in sub_group:
                         if not line.startswith('Relative dose') and len(line.split()) == 3 :     # 3 cols of data                        
                            aa = [float(i) for i in line.split()]                       
                            #data = np.asarray(data) 
                            data.append(aa)
                
            
                else:
                    info = {};
                    for line in sub_group: 
                        if len(line.split()) == 2 : 
                            field,value=line.split(':')
                            if isinstance(value, str): # if a string
                                value=value.strip()
                            else:
                                value = float(value)
                            info[field]=value
    
    i = i + 1 


0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [299]:
from decimal import *
line = '                0                   0                       100'
print(len(line.split()))
bb = [float(i) for i in line.split()]
bb
#results =[float(i) for i in bb]

#results

3


[0.0, 0.0, 100.0]